# Slicing

Data in a [Variable](../generated/scipp.Variable.rst#scipp.Variable) or [Dataset](../generated/scipp.Dataset.rst#scipp.Dataset) can be indexed in a similar manner to NumPy and xarray.
The dimension to be sliced is specified using a dimension label and, in contrast to NumPy, positional dimension lookup is not available.
Positional indexing with an integer or an integer range is using `__getitem__` and `__setitem__` with a dimension label as first argument.
This is available for variables, datasets, as well as items of a dataset.
In all cases a *view* is returned, i.e., just like when slicing a [numpy.ndarray](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html#numpy.ndarray) no copy is performed.

Consider the following variable:

In [1]:
import numpy as np
import scipp as sc

var = sc.Variable(
    dims=['z', 'y', 'x'],
    values=np.random.rand(2, 3, 4),
    variances=np.random.rand(2, 3, 4))
sc.show(var)

As when slicing a `numpy.ndarray`, the dimension `'x'` is removed since no range is specified:

In [2]:
s = var['x', 1]
sc.show(s)
print(s.dims, s.shape)

[z, y] [2, 3]


When a range is specified, the dimension is kept, even if it has extent 1:

In [3]:
s = var['x', 1:3]
sc.show(s)
print(s.dims, s.shape)

s = var['x', 1:2]
sc.show(s)
print(s.dims, s.shape)

[z, y, x] [2, 3, 2]


[z, y, x] [2, 3, 1]


Slicing can be chained arbitrarily:

In [4]:
s = var['x', 1:4]['y', 2]['x', 1]
sc.show(s)
print(s.dims, s.shape)

[z] [2]


Slicing for datasets works in the same way, but some additional rules apply:

In [5]:
d = sc.Dataset(
    {'a': sc.Variable(dims=['x', 'y'], values=np.random.rand(2, 3)),
     'b': sc.Variable(dims=['y', 'x'], values=np.random.rand(3, 2)),
     'c': sc.Variable(dims=['x'], values=np.random.rand(2)),
     '0d-data': sc.Variable(1.0)},
    coords={
        'x': sc.Variable(['x'], values=np.arange(2.0), unit=sc.units.m),
        'y': sc.Variable(['y'], values=np.arange(3.0), unit=sc.units.m),
        'aux_x': sc.Variable(['x'], values=np.arange(2.0), unit=sc.units.m),
        'aux_y': sc.Variable(['y'], values=np.arange(3.0), unit=sc.units.m)})
sc.show(d)

As when slicing a variable, the sliced dimension is removed when slicing without range, and kept when slicing with range.

When slicing a dataset a number of other things happen as well:

- Any data item that does not depend on the sliced dimension is removed.
- Slicing **without range**:
  - The *coordinates* for the sliced dimension are *removed*.
- Slicing **with a range**:
  - The *coordinates* for the sliced dimension are *kept*.


This is an important aspect and it is worthwhile to take some time and think through the mechanism.
Consider the following example, contrasting slicing with and without range:

- We slice dimension `'x'`, so the data item `'0d-data'` which does not depend on dimension `'x'` is not visible in the slice views.
- In the second case (without range) the coord for dimension `'x'` is also not part of the slice view

Make sure to inspect the `dims` and `shape` of all variable (data and coordinates) of the resulting slice views (note the tooltip shown when moving the mouse over the name also contains this information):

In [6]:
# Range of length 1
sc.show(d['x', 1:2])
d['x', 1:2]

<scipp.DatasetView>
Dimensions: {{x, 1}, {y, 3}}
Coordinates:
    y                         float64    [m]              (y)  [0.000000, 1.000000, 2.000000]
    x                         float64    [m]              (x)  [1.000000]
    aux_y                     float64    [m]              (y)  [0.000000, 1.000000, 2.000000]
    aux_x                     float64    [m]              (x)  [1.000000]
Data:
    a                         float64    [dimensionless]  (x, y)  [0.683385, 0.471514, 0.895528]
    b                         float64    [dimensionless]  (y, x)  [0.039433, 0.422479, 0.320092]
    c                         float64    [dimensionless]  (x)  [0.659236]

In [7]:
# No range
sc.show(d['x', 1])
d['x', 1]

<scipp.DatasetView>
Dimensions: {{y, 3}}
Coordinates:
    y                         float64    [m]              (y)  [0.000000, 1.000000, 2.000000]
    aux_y                     float64    [m]              (y)  [0.000000, 1.000000, 2.000000]
Data:
    a                         float64    [dimensionless]  (y)  [0.683385, 0.471514, 0.895528]
    b                         float64    [dimensionless]  (y)  [0.039433, 0.422479, 0.320092]
    c                         float64    [dimensionless]  ()  [0.659236]

Slicing a data item of a dataset should not bring any surprises.
Essentially this behaves like slicing a dataset with just a single data item:

In [8]:
sc.show(d['a']['x', 1:2])

Slicing and item access can be done in arbitrary order with identical results:

In [9]:
d['x', 1:2]['a'] == d['a']['x', 1:2]
d['x', 1:2]['a'].coords['x'] == d.coords['x']['x', 1:2]

True